In [14]:
import numpy as np
import lsst.daf.butler as dafButler
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import sys


In [15]:

repo = '/sdf/group/rubin/repo/embargo/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry
print("Butler successfuly instatiated")
collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
#collection = sys.argv[1]


Butler successfuly instatiated


In [6]:

def test_for_calibration(butler, collection, calibration_type) :
    ## This function just tries to get a defect from the collection.
    ## If successfull it passes True, otherwise False.
    try :
        butler.get(calibration_type, instrument='LATISS', physical_filter='SDSSr_65mm~empty', detector=0, collections=collection)
        exists = True
    except :
        exists = False
    return exists

butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry
#####
def get_chain(collection):
    chain = registry.getCollectionChain(collection)
    calibs = [c for c in sorted(chain, reverse = True) if "LATISS/calib" in c]
    return calibs


In [7]:

calib_chain = get_chain(collection)
"""with open(f'{collection}_calibrations_chained.txt', 'w') as f:
    for col in calib_chain:
        f.write(f"{col}\n")"""
print("Chained calibrations are the following :")
for c in calib_chain:
    print(c)
print("\n\n")
#####
c_types = np.array(["flat-i", "flat-z", "flat-y", "flat-g", "flat-r", "bias", "defects", "dark", "crosstalk"])

calib_types, calib_names = [], []
for i, element in enumerate(calib_chain):
    for calib_t in c_types:
        if calib_t in element and calib_t not in calib_types:
            print(f"Found {calib_t}.")
            calib_types.append(calib_t)
            calib_names.append(element)
            """
with open(f'{collection}_last_calibrations.txt', 'w') as f:
    for col1, col2 in np.column_stack((calib_names, calib_types)):
        f.write(f"{col1}\t{col2}\n")"""

Chained calibrations are the following :
LATISS/calib/DM-41633/flat-i
LATISS/calib/DM-41319/unbounded
LATISS/calib/DM-41319
LATISS/calib/DM-40904//flat-r
LATISS/calib/DM-40904//flat-i
LATISS/calib/DM-40904//flat-g
LATISS/calib/DM-40904//defects
LATISS/calib/DM-40904//dark
LATISS/calib/DM-40904//bias
LATISS/calib/DM-39505/crosstalk.20230602
LATISS/calib/DM-38946/noRGseq/flat-r.20230503
LATISS/calib/DM-38946/noRGseq/flat-i.20230503
LATISS/calib/DM-38946/noRGseq/flat-g.20230503
LATISS/calib/DM-38946/noRGseq/dark.20230503
LATISS/calib/DM-38946/noRGseq/bias.20230503
LATISS/calib/DM-37587/flat-SDSSr.20230113a
LATISS/calib/DM-37587/flat-OG550.20230113a
LATISS/calib/DM-37587/flat-BG40.20230113a
LATISS/calib/DM-36719/flat-SDSSr.20221107
LATISS/calib/DM-36719/flat-SDSSi.20221107
LATISS/calib/DM-36719/flat-SDSSg.20221107
LATISS/calib/DM-36719/dark.20221107
LATISS/calib/DM-36719/bias.20221107
LATISS/calib/DM-36484/flat-SDSSr.20221006a
LATISS/calib/DM-36484/flat-SDSSi.20221006a
LATISS/calib/DM-3648

In [9]:
datasets = []
coll_paths = []
physical_filters = []
for cal_n, cal_t in np.column_stack((calib_names, calib_types)):
    print(cal_n, cal_t)
    if "flat" in cal_t:
        dataset = list(registry.queryDatasets("flat", collections = cal_n))
        physical_filters.append(dataset[0].dataId["physical_filter"])
    else :
        dataset = list(registry.queryDatasets(cal_t, collections = cal_n))
        physical_filters.append(0) #For convenient saving later
    datasets.append(dataset)
    coll_paths.append(dataset[0].run)
print("\n", datasets, "\n\n", coll_paths, "\n\n", physical_filters)
"""with open(f'{collection}_calibrations_access.txt', 'w') as f:
    for col1, col2 in np.column_stack((coll_paths, physical_filters)):
        f.write(f"{col1}\t{col2}\n")"""

LATISS/calib/DM-41633/flat-i flat-i
LATISS/calib/DM-40904//flat-r flat-r
LATISS/calib/DM-40904//flat-g flat-g
LATISS/calib/DM-40904//defects defects
LATISS/calib/DM-40904//dark dark
LATISS/calib/DM-40904//bias bias
LATISS/calib/DM-39505/crosstalk.20230602 crosstalk

 [[DatasetRef(DatasetType('flat', {band, instrument, detector, physical_filter}, ExposureF, isCalibration=True), {instrument: 'LATISS', detector: 0, physical_filter: 'empty~SDSSi_65mm', band: 'i'}, run='LATISS/calib/DM-41633/flatGen-i.20231107a/20231107T201619Z', id=a0279de4-c2a1-4bd7-8d68-1a1d49a7191b)], [DatasetRef(DatasetType('flat', {band, instrument, detector, physical_filter}, ExposureF, isCalibration=True), {instrument: 'LATISS', detector: 0, physical_filter: 'SDSSr_65mm~empty', band: 'r'}, run='LATISS/calib/DM-40904/cleaned/flatGen-r.20230925a/20230925T221124Z', id=d4dd4c74-f061-417b-a753-d210733b027f)], [DatasetRef(DatasetType('flat', {band, instrument, detector, physical_filter}, ExposureF, isCalibration=True), {i

'with open(f\'{collection}_calibrations_access.txt\', \'w\') as f:\n    for col1, col2 in np.column_stack((coll_paths, physical_filters)):\n        f.write(f"{col1}\t{col2}\n")'

In [17]:
calib_names, calib_types = np.loadtxt(f"{collection.split('/')[-1]}_last_calibrations.txt", dtype = 'str', delimiter = '\t',unpack=True)

In [19]:
calib_names

array(['LATISS/calib/DM-41633/flat-i', 'LATISS/calib/DM-40904//flat-r',
       'LATISS/calib/DM-40904//flat-g', 'LATISS/calib/DM-40904//defects',
       'LATISS/calib/DM-40904//dark', 'LATISS/calib/DM-40904//bias',
       'LATISS/calib/DM-39505/crosstalk.20230602'], dtype='<U40')

In [24]:
np.loadtxt("PREOPS-4553_raw_data_calibration.txt",dtype="array")

TypeError: data type 'array' not understood

In [2]:
import numpy as np

In [4]:
collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
calib_names, calib_types = np.loadtxt(f"{collection.split('/')[-1]}_last_calibrations.txt", dtype = 'str', delimiter = '\t',unpack=True)

In [11]:
np.loadtxt(f"{collection.split('/')[-1]}_data_calibration.txt",dtype="float")

ValueError: could not convert string '[[0.5572045' to float64 at row 0, column 1.

In [12]:
with open(f"{collection.split('/')[-1]}_data_calibration.txt", 'r') as file:
    arrays = [eval(line.strip()) for line in file]

# Display the arrays
for arr in arrays:
    print(arr)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)

In [16]:
calib_names, calib_types = np.loadtxt(f"{collection.split('/')[-1]}_last_calibrations.txt", dtype = 'str', delimiter = '\t',unpack=True)
coll_paths, physical_filters = np.loadtxt(f"{collection.split('/')[-1]}_calibrations_access.txt", dtype = 'str', delimiter = '\t',unpack=True)

calib_arr = [[] for i in range(len(coll_paths))]
for i, cal_n in enumerate(coll_paths):
    if "flat" not in calib_types[i] :     
        file = butler.get(calib_types[i], instrument='LATISS',detector=0, collections=cal_n)
        if calib_types[i]=="bias" or calib_types[i]=="dark":
            calib_arr[i] = file.getImage().getArray()
        if calib_types[i]=="defects" :
            print("defects!")
            calib_arr[i] = file.toTable()[0]
        if calib_types[i]=="crosstalk":
            print("crosstalk!")
            calib_arr[i] = file.coeffs
    else : 
        file = butler.get("flat", instrument='LATISS', physical_filter=physical_filters[i] , detector=0, collections=cal_n)
        calib_arr[i] = file.getImage().getArray()



defects = calib_arr[np.where(calib_types=="defects")[0][0]]
msk = np.ones((4000,4072))
for defect in defects :
    x0  = defect['x0']
    y0  = defect['y0']
    width  = defect['width']
    height  = defect['height']
    for x in range(x0, x0 + width):
        for y in range(y0, y0 + height):
            msk[y][x] = 0
    calib_arr[np.where(calib_types=="defects")[0][0]]=msk

defects!
crosstalk!


In [18]:
from astropy.table import Table

In [43]:
dict = {"calib_data":calib_arr, "calib_names":calib_types}

In [44]:
import pickle

# Your dictionary

# Save the dictionary to a file using pickle
with open('my_dict_test.pkl', 'wb') as file:
    pickle.dump(dict, file)

In [46]:
with open('my_dict_test.pkl', 'rb') as file:
    loaded_dict = pickle.load(file)

# Display the loaded dictionary
print(loaded_dict)

{'calib_data': [array([[0.5572045 , 0.5446379 , 0.54021287, ..., 0.5387925 , 0.5418706 ,
        0.55454063],
       [0.7214014 , 0.71764904, 0.7174892 , ..., 0.6972188 , 0.707039  ,
        0.6988728 ],
       [0.7837816 , 0.79516584, 0.79410744, ..., 0.7748249 , 0.77971786,
        0.757288  ],
       ...,
       [0.7926851 , 0.7957241 , 0.8039826 , ..., 0.8150578 , 0.79769206,
        0.7882936 ],
       [0.71867424, 0.71907014, 0.72177416, ..., 0.7314326 , 0.7372414 ,
        0.72408205],
       [0.56404746, 0.5552264 , 0.5475206 , ..., 0.56067485, 0.5628423 ,
        0.5809347 ]], dtype=float32), array([[0.53118426, 0.519401  , 0.51501966, ..., 0.523724  , 0.526197  ,
        0.5394344 ],
       [0.691123  , 0.68724287, 0.6884574 , ..., 0.6790806 , 0.6878154 ,
        0.68083817],
       [0.755341  , 0.76125145, 0.7640907 , ..., 0.75186515, 0.7645376 ,
        0.7465358 ],
       ...,
       [0.79438263, 0.79894364, 0.80488753, ..., 0.821287  , 0.80251116,
        0.7927004 ],
   

In [19]:
summary_table = Table()#"calib_name":calib_names, "calib_type":calib_types, 

In [29]:
#calib_arr[7]

In [20]:
summary_table

calib_data
object
[[0.5572045 0.5446379 0.54021287 ... 0.5387925 0.5418706 0.55454063]\n [0.7214014 0.71764904 0.7174892 ... 0.6972188 0.707039 0.6988728 ]\n [0.7837816 0.79516584 0.79410744 ... 0.7748249 0.77971786 0.757288 ]\n ...\n [0.7926851 0.7957241 0.8039826 ... 0.8150578 0.79769206 0.7882936 ]\n [0.71867424 0.71907014 0.72177416 ... 0.7314326 0.7372414 0.72408205]\n [0.56404746 0.5552264 0.5475206 ... 0.56067485 0.5628423 0.5809347 ]]
[[0.53118426 0.519401 0.51501966 ... 0.523724 0.526197 0.5394344 ]\n [0.691123 0.68724287 0.6884574 ... 0.6790806 0.6878154 0.68083817]\n [0.755341 0.76125145 0.7640907 ... 0.75186515 0.7645376 0.7465358 ]\n ...\n [0.79438263 0.79894364 0.80488753 ... 0.821287 0.80251116 0.7927004 ]\n [0.72042406 0.71971893 0.7221339 ... 0.735372 0.738317 0.7286132 ]\n [0.56558037 0.55305445 0.5434498 ... 0.5617544 0.56609994 0.5835881 ]]
[[0.5260515 0.5139311 0.50995296 ... 0.5309905 0.5298917 0.5407272 ]\n [0.6835112 0.6787517 0.6790853 ... 0.6834096 0.6931233 0.6825336 ]\n [0.74341047 0.7533898 0.7516927 ... 0.7511993 0.78599393 0.7658935 ]\n ...\n [0.78820646 0.79095745 0.7959417 ... 0.83513594 0.81707066 0.80618864]\n [0.71549255 0.71323705 0.71430266 ... 0.74626124 0.754323 0.7410096 ]\n [0.5611995 0.5487554 0.5415498 ... 0.5714623 0.5767249 0.59344417]]
[[1. 1. 1. ... 1. 1. 1.]\n [1. 1. 1. ... 1. 1. 1.]\n [1. 1. 1. ... 1. 1. 1.]\n ...\n [1. 1. 1. ... 1. 1. 1.]\n [1. 1. 1. ... 1. 1. 1.]\n [1. 1. 1. ... 1. 1. 1.]]
[[-0.04168844 0.01879784 0.08123881 ... -0.05567029 0.05135338\n -0.13567556]\n [-0.08550237 0.15162966 -0.03988761 ... 0.02317207 0.00714302\n -0.05432408]\n [ 0.08084349 -0.02434086 0.13498765 ... 0.0096118 0.04453782\n -0.05088754]\n ...\n [ 0.01223799 -0.04325746 0.01373272 ... -0.03076998 -0.02112245\n -0.01964985]\n [-0.10376193 0.01317381 0.01292841 ... -0.03830409 -0.06980523\n -0.02844927]\n [-0.02564387 -0.05765276 0.04320485 ... -0.08425498 0.03273935\n -0.04208207]]
[[-0.863276 0.2358092 -1.9689058 ... 0.1526779 -0.99787\n 0.45491633]\n [ 1.4115597 -2.9874005 0.9947448 ... -0.02571768 -0.6025105\n 0.41385287]\n [ 0.02563414 -0.85411876 -1.0665271 ... -0.12480616 -1.4756793\n -1.6286763 ]\n ...\n [ 0.37739453 1.0246592 0.56357753 ... 0.0176957 -0.2863406\n -1.4429235 ]\n [ 2.5952399 -0.48002797 0.533927 ... 0.28111273 0.9175564\n -0.6560611 ]\n [-0.7528054 1.2927809 -1.1560043 ... 1.4664497 -0.3218582\n 0.19577381]]
[[0.00000000e+00 1.35567350e-04 2.12072266e-05 8.52183217e-06\n 3.48150040e-07 3.49318197e-06 1.77419743e-06 0.00000000e+00\n 0.00000000e+00 0.00000000e+00 6.62398419e-08 0.00000000e+00\n 0.00000000e+00 0.00000000e+00 2.86405942e-06 3.57817430e-05]\n [2.06497854e-04 0.00000000e+00 2.20243043e-04 5.79033171e-05\n 4.59381003e-05 2.80140977e-05 1.18616260e-05 1.63705736e-06\n 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n 0.00000000e+00 2.83113295e-06 0.00000000e+00 2.66771957e-05]\n [3.83107169e-05 2.39972276e-04 0.00000000e+00 2.17237245e-04\n 6.14284722e-05 5.07460819e-05 6.38135915e-06 1.14019232e-06\n 1.66811500e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00\n 1.04952278e-05 8.24884804e-07 1.44881867e-05 0.00000000e+00]\n [3.54888816e-05 3.40231367e-05 3.58520001e-04 0.00000000e+00\n 1.35303662e-04 3.30329719e-05 7.41672241e-06 0.00000000e+00\n 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n 3.97106818e-07 2.29065423e-05 7.45236037e-06 0.00000000e+00]\n [1.28023286e-05 4.17098182e-05 4.52879926e-05 1.91586800e-04\n 0.00000000e+00 2.75243992e-04 0.00000000e+00 4.58455352e-06\n 0.00000000e+00 1.19626703e-05 4.36748529e-05 0.00000000e+00\n 0.00000000e+00 7.69800868e-06 0.00000000e+00 0.00000000e+00]\n [3.59299484e-06 2.32679944e-05 4.22184379e-05 1.35286551e-05\n 8.62249304e-04 0.00000000e+00 1.08008204e-04 7.52286553e-06\n 2.69440219e-06 5.26742820e-06 1.10848610e-05 2.34962119e-05\n 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.94339146e-06]\n [4.00179801e-07 4.41571374e-06 4.38923960e-06 0.00000000e+00\n 0.00000000e+00 2.49640573e-04 